# PROBLEM 1: Topic Models
## Obtain Topic Models (K=10, 20, 50) for both datasets by running LDA and NMF methods; you can call libraries for both methods and dont have to use the ES index as source. For both LDA and NMF: print out for each topic the top 20 words (with probabilities)

The rest of of topic exercises and results are required only for the LDA topics:

- 20NG: how well the topics align with the 20NG label classes? This is not asking for a measurement, but rather for a visual inspection to determine what topics match well with what classes. 

Does this change if one increases the topics from 20 to 50?


### 1. Data Preprocessing using gensim

In [1]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-download-auto-examples-tutorials-run-lda-py

# imports

from sklearn.datasets import fetch_20newsgroups
from gensim import corpora, models, similarities, downloader

# Load the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# truncate the dataset to 1000 documents for faster processing
# newsgroups.data = newsgroups.data[:1000]

print(type(newsgroups.data[0][:]))  # <class 'scipy.sparse.csr.csr_matrix'>
print(newsgroups.data[2][:])  # <class 'numpy.ndarray'>



<class 'str'>
well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk rather than the 120, but i don't really have
a feel for how much "better" the display is (yea, it looks great in the
store, but is that all "wow" or i

In [2]:
# gensim expects a list of lists of tokens, so we need to convert the sparse matrix to a list of lists
# %pip install nltk
# Tokenize the documents
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

for idx in range(len(newsgroups.data)):
    newsgroups.data[idx] = newsgroups.data[idx][:].lower()  # convert to lowercase
    newsgroups.data[idx] = tokenizer.tokenize(newsgroups.data[idx])  # split into words

print(newsgroups.data[2]) 
print(len(newsgroups.data[2])) # number of tokens in the document

['well', 'folks', 'my', 'mac', 'plus', 'finally', 'gave', 'up', 'the', 'ghost', 'this', 'weekend', 'after', 'starting', 'life', 'as', 'a', '512k', 'way', 'back', 'in', '1985', 'sooo', 'i', 'm', 'in', 'the', 'market', 'for', 'a', 'new', 'machine', 'a', 'bit', 'sooner', 'than', 'i', 'intended', 'to', 'be', 'i', 'm', 'looking', 'into', 'picking', 'up', 'a', 'powerbook', '160', 'or', 'maybe', '180', 'and', 'have', 'a', 'bunch', 'of', 'questions', 'that', 'hopefully', 'somebody', 'can', 'answer', 'does', 'anybody', 'know', 'any', 'dirt', 'on', 'when', 'the', 'next', 'round', 'of', 'powerbook', 'introductions', 'are', 'expected', 'i', 'd', 'heard', 'the', '185c', 'was', 'supposed', 'to', 'make', 'an', 'appearence', 'this', 'summer', 'but', 'haven', 't', 'heard', 'anymore', 'on', 'it', 'and', 'since', 'i', 'don', 't', 'have', 'access', 'to', 'macleak', 'i', 'was', 'wondering', 'if', 'anybody', 'out', 'there', 'had', 'more', 'info', 'has', 'anybody', 'heard', 'rumors', 'about', 'price', 'drops

In [3]:
# Remove numbers, but not words that contain numbers.
for ng in newsgroups.data:
    ng = [token for token in ng if not token.isdigit()]
    # Remove words that are only one character.
    ng = [token for token in ng if len(token) > 1]



print(newsgroups.data[2])  
print(len(newsgroups.data[2]))

['well', 'folks', 'my', 'mac', 'plus', 'finally', 'gave', 'up', 'the', 'ghost', 'this', 'weekend', 'after', 'starting', 'life', 'as', 'a', '512k', 'way', 'back', 'in', '1985', 'sooo', 'i', 'm', 'in', 'the', 'market', 'for', 'a', 'new', 'machine', 'a', 'bit', 'sooner', 'than', 'i', 'intended', 'to', 'be', 'i', 'm', 'looking', 'into', 'picking', 'up', 'a', 'powerbook', '160', 'or', 'maybe', '180', 'and', 'have', 'a', 'bunch', 'of', 'questions', 'that', 'hopefully', 'somebody', 'can', 'answer', 'does', 'anybody', 'know', 'any', 'dirt', 'on', 'when', 'the', 'next', 'round', 'of', 'powerbook', 'introductions', 'are', 'expected', 'i', 'd', 'heard', 'the', '185c', 'was', 'supposed', 'to', 'make', 'an', 'appearence', 'this', 'summer', 'but', 'haven', 't', 'heard', 'anymore', 'on', 'it', 'and', 'since', 'i', 'don', 't', 'have', 'access', 'to', 'macleak', 'i', 'was', 'wondering', 'if', 'anybody', 'out', 'there', 'had', 'more', 'info', 'has', 'anybody', 'heard', 'rumors', 'about', 'price', 'drops

In [4]:
%pip install nltk
import nltk
# nltk.download()

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\koola\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
# Lemmatize the documents
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
newsgroups.data = [[lemmatizer.lemmatize(token) for token in doc] for doc in newsgroups.data]

In [6]:
# Compute bigrams
from gensim.models import Phrases
bigrams = Phrases(newsgroups.data, min_count=5, threshold=100)
# bigrams = Phrases(newsgroups.data, min_count=20)
for idx in range(len(newsgroups.data)):
    for token in bigrams[newsgroups.data[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            newsgroups.data[idx].append(token)

In [7]:
# Removing words that appear in less than 20 docs OR in more than 50% of the docs.

# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(newsgroups.data)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
# dictionary.filter_extremes(no_below=20, no_above=0.5)

In [8]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in newsgroups.data]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 98832
Number of documents: 11314


### 2. Modeling

In [9]:
# We are ready to train the LDA model. We will first discuss how to set some of the training parameters.

# chunksize 
    # controls how many documents are processed at a time in the training algorithm. 
    # Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. 
    # I’ve set chunksize = 2000, which is more than the amount of documents, so I process all the data in one go. 
    # Chunksize can however influence the quality of the model, as discussed in Hoffman and co-authors [2], but the difference was not substantial in this case.
# passes
    # controls how often we train the model on the entire corpus. 
    # Another word for passes might be “epochs”. iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document.
    # It is important to set the number of “passes” and “iterations” high enough.

from gensim.models import LdaModel

# Raghu will probably ask what params i am passing to the model
# Set training parameters.
num_topics = 20
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

# Train the model.
model = LdaModel(corpus=corpus, 
                 id2word=id2word, 
                 chunksize=chunksize, 
                 passes=passes, 
                 iterations=iterations, 
                 num_topics=num_topics, 
                 eval_every=eval_every)


In [10]:
# Print the Keyword in the topics -> K = 10
topics = model.print_topics(num_topics=10 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.146*"_" + 0.026*"msg" + 0.010*"lost" + 0.008*"mono" + 0.006*"voluntary" + 0.005*"glutamate" + 0.005*"nissan" + 0.002*"61" + 0.001*"double_blind" + 0.001*"bean" + 0.001*"au" + 0.001*"mq" + 0.001*"ph" + 0.001*"sodium" + 0.001*"stanza" + 0.001*"jr" + 0.000*"altima" + 0.000*"_______________________________________________________________________________" + 0.000*"johnh" + 0.000*"mpce"')
(18, '0.055*"game" + 0.050*"team" + 0.026*"player" + 0.021*"hockey" + 0.021*"year" + 0.020*"season" + 0.017*"play" + 0.014*"win" + 0.013*"nhl" + 0.012*"he" + 0.012*"fan" + 0.009*"last" + 0.008*"run" + 0.008*"blue" + 0.008*"pick" + 0.008*"baseball" + 0.007*"playoff" + 0.007*"hit" + 0.007*"cup" + 0.007*"city"')
(19, '0.044*"i" + 0.038*"the" + 0.035*"a" + 0.027*"to" + 0.020*"it" + 0.018*"and" + 0.017*"is" + 0.016*"for" + 0.013*"you" + 0.012*"have" + 0.012*"with" + 0.011*"on" + 0.010*"this" + 0.010*"if" + 0.010*"can" + 0.009*"in" + 0.009*"that" + 0.008*"or" + 0.008*"do" + 0.007*"drive"')
(8, '0.020*"hz" 

In [11]:
# -> K = 20
topics = model.print_topics(num_topics=20 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.146*"_" + 0.026*"msg" + 0.010*"lost" + 0.008*"mono" + 0.006*"voluntary" + 0.005*"glutamate" + 0.005*"nissan" + 0.002*"61" + 0.001*"double_blind" + 0.001*"bean" + 0.001*"au" + 0.001*"mq" + 0.001*"ph" + 0.001*"sodium" + 0.001*"stanza" + 0.001*"jr" + 0.000*"altima" + 0.000*"_______________________________________________________________________________" + 0.000*"johnh" + 0.000*"mpce"')
(1, '0.122*"00" + 0.009*"cover" + 0.007*"penalty" + 0.004*"warrior" + 0.004*"bid" + 0.004*"overtime" + 0.004*"mutant" + 0.003*"richer" + 0.003*"njd" + 0.003*"barrasso" + 0.002*"marvel" + 0.002*"archer" + 0.002*"bagged" + 0.002*"semak" + 0.002*"samuelsson" + 0.002*"scum" + 0.002*"niedermayer" + 0.002*"penalty_shot" + 0.002*"tippett" + 0.002*"tripping"')
(2, '0.008*"swiss" + 0.007*"candida" + 0.006*"yeast" + 0.005*"leaning" + 0.004*"allergic" + 0.002*"albicans" + 0.002*"candida_albicans" + 0.002*"characterization" + 0.001*"bezier" + 0.001*"curve" + 0.001*"leaning_towards" + 0.001*"bezier_curve" + 0.001

In [12]:
# -> K = 50
topics = model.print_topics(num_topics=50 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.146*"_" + 0.026*"msg" + 0.010*"lost" + 0.008*"mono" + 0.006*"voluntary" + 0.005*"glutamate" + 0.005*"nissan" + 0.002*"61" + 0.001*"double_blind" + 0.001*"bean" + 0.001*"au" + 0.001*"mq" + 0.001*"ph" + 0.001*"sodium" + 0.001*"stanza" + 0.001*"jr" + 0.000*"altima" + 0.000*"_______________________________________________________________________________" + 0.000*"johnh" + 0.000*"mpce"')
(1, '0.122*"00" + 0.009*"cover" + 0.007*"penalty" + 0.004*"warrior" + 0.004*"bid" + 0.004*"overtime" + 0.004*"mutant" + 0.003*"richer" + 0.003*"njd" + 0.003*"barrasso" + 0.002*"marvel" + 0.002*"archer" + 0.002*"bagged" + 0.002*"semak" + 0.002*"samuelsson" + 0.002*"scum" + 0.002*"niedermayer" + 0.002*"penalty_shot" + 0.002*"tippett" + 0.002*"tripping"')
(2, '0.008*"swiss" + 0.007*"candida" + 0.006*"yeast" + 0.005*"leaning" + 0.004*"allergic" + 0.002*"albicans" + 0.002*"candida_albicans" + 0.002*"characterization" + 0.001*"bezier" + 0.001*"curve" + 0.001*"leaning_towards" + 0.001*"bezier_curve" + 0.001

In [ ]:
# - 20NG: how well the topics align with the 20NG label classes? This is not asking for a measurement, but rather for a visual inspection to determine what topics match well with what classes. Does this change if one increases the topics from 20 to 50?
i = 0
labels = set(newsgroups.target)
# print(newsgroups.target_names)
for label in newsgroups.target_names:
    print(i, ": ", newsgroups.target_names[i])
    i += 1
# these do not correlate, sort them or use a dictionary to align them correctly. 

0 :  alt.atheism
1 :  comp.graphics
2 :  comp.os.ms-windows.misc
3 :  comp.sys.ibm.pc.hardware
4 :  comp.sys.mac.hardware
5 :  comp.windows.x
6 :  misc.forsale
7 :  rec.autos
8 :  rec.motorcycles
9 :  rec.sport.baseball
10 :  rec.sport.hockey
11 :  sci.crypt
12 :  sci.electronics
13 :  sci.med
14 :  sci.space
15 :  soc.religion.christian
16 :  talk.politics.guns
17 :  talk.politics.mideast
18 :  talk.politics.misc
19 :  talk.religion.misc


In [13]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -6.7917.
[([(0.054705698, 'the'),
   (0.031596676, 'to'),
   (0.03011224, 'a'),
   (0.023963792, 'of'),
   (0.023823874, 'that'),
   (0.023553543, 'i'),
   (0.021047836, 'and'),
   (0.018114466, 'it'),
   (0.017901864, 'is'),
   (0.016149461, 'in'),
   (0.015000125, 'you'),
   (0.010061382, 'not'),
   (0.009536543, 't'),
   (0.008867833, 'have'),
   (0.008862376, 'they'),
   (0.00841737, 's'),
   (0.008335884, 'this'),
   (0.008141373, 'wa'),
   (0.0077865203, 'be'),
   (0.0077219834, 'are')],
  -0.5991151638209106),
 ([(0.07364785, 'the'),
   (0.035641737, 'of'),
   (0.031224828, 'a'),
   (0.028678423, 'and'),
   (0.027896354, 'to'),
   (0.021217577, 'in'),
   (0.015873805, 'for'),
   (0.015675051, 'is'),
   (0.009537798, 'be'),
   (0.009030413, 'on'),
   (0.008356059, 'are'),
   (0.0077500017, 'or'),
   (0.0072281784, 'with'),
   (0.0070184725, 'this'),
   (0.0068705697, 'by'),
   (0.0061281584, 's'),
   (0.0057291547, 'will'),
   (0.005203582, 'from'),
   (0

In [21]:
# NMF: Idea of the algorithm is as follows:

# Init W, A and B matrices

# Input corpus
# Split corpus into batches

# for v in batches: 
    # infer h:
        # do coordinate gradient descent step to find h that minimizes (v - Wh) l2 norm
        # bound h so that it is non-negative

    # update A and B:
        # A = h.dot(ht)
        # B = v.dot(ht)

    # update W:
        # do gradient descent step to find W that minimizes 0.5*trace(W^t W A) - trace (W^t B) l2 norm